# "Classifiez automatiquement des biens de consommation"
_NLP Cleaning Notebook_

## 0 Preliminaries

### 0.0 Importing Packages and Modules

Checking whether the notebook is on Colab or PC

In [1]:
import sys
is_colab = 'google.colab' in sys.modules
is_colab, sys.executable

(True, '/usr/bin/python3')

Mounting my Drive if on Colab

In [2]:
if is_colab==True:
    from google.colab import files, output, drive
    drive.mount('/gdrive')
    %cd /gdrive
    print("You're on Google Colab")
else:
    print("You're on a PC")

Mounted at /gdrive
/gdrive
You're on Google Colab


Installations and importations required in the virtual environment.

In [3]:
# import os
# if os.getcwd()!='/gdrive/My Drive/--DATA SCIENCE/PROJET6/NOTEBOOKS':
#     os.chdir('/gdrive/My Drive/--DATA SCIENCE/PROJET6/NOTEBOOKS')

In [ ]:
import os
if is_colab==True:
    if os.getcwd()!='/gdrive/My Drive/--DATA SCIENCE/PROJET6/NOTEBOOKS':
        os.chdir('/gdrive/My Drive/--DATA SCIENCE/PROJET6/NOTEBOOKS')
else:
    if not (os.path.exists(os.getcwd()+'/requirements_nlp.txt') \
                     and os.path.exists(os.getcwd()+'/P6_functions.py')):
        print("ERROR: Make sure 'P6_functions.py' and \
'requirements_nlp.txt' are in the current working directory")

!pip install -r requirements_nlp.txt

     |████████████████████████████████| 6.5MB 2.4MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 5.3MB 37.6MB/s 
     |████████████████████████████████| 7.8MB 46.8MB/s 
     |████████████████████████████████| 163kB 50.4MB/s 
     |████████████████████████████████| 3.8MB 40.9MB/s 
     |████████████████████████████████| 1.6MB 55.1MB/s 
     |████████████████████████████████| 460kB 40.0MB/s 
     |████████████████████████████████| 31.9MB 159kB/s 
     |████████████████████████████████| 307kB 52.4MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 


In [ ]:
from P6_functions import *

Installations (creating the requirements file)

In [ ]:
# in order to get orca to be installed in colab (to display static plotly graphs)
!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [ ]:
# !pip install gtts
# !pip install wikipedia2vec==0.2.2

In [ ]:
# !pip freeze > requirements_cleaning_nlp.txt

Importation of modules and packages. 

In [ ]:
import io

import string

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor']='w'

# import warnings
# warnings.filterwarnings('ignore')

Setting pandas display options.

In [ ]:
dictPdSettings = {'display.max_rows': 500, 'display.width': 100,
                  'display.max_colwidth': 100,
                  'display.float_format': lambda x: '%.2f' % x}
for k,v in dictPdSettings.items():
  pd.set_option(k,v)

To play audio text-to-speech during execution.

In [ ]:
from IPython.display import Audio
from gtts import gTTS

def speak(text, lang='en'):
    with io.BytesIO() as f:
        gTTS(text=text, lang=lang).write_to_fp(f)
        f.seek(0)
        return Audio(f.read(), autoplay=True)

In [ ]:
speak('Packages and modules successfully imported')

### 0.1 Importing the datasets

Data is composed of 9 distinct .csv files we'll load in a dictionnary of dataframes.

In [ ]:
if is_colab==True:
    # Importing database from my Drive
    print("Try to import data files in the notebook from myDrive...")
else:
    # Importing database from PC
    print("Try to import data files in the notebook from PC ('DATA')...")

df = pd.read_csv("../DATA/flipkart_com-ecommerce_sample_1050.csv",
                 sep=',', 
                 index_col = 'uniq_id',
                 encoding ='utf-8')

print("-----> Importation of .csv in the notebook: OK")

In [ ]:
speak('Datasets successfully imported')

### 0.2 First Overview

In [ ]:
df.describe(include='all')

Printing total nb and percentage of null:

In [ ]:
display(print_null_pct(df))

In [ ]:
df.isna().sum()

Browsing the content

In [ ]:
df[df.isna().any(1)].sample(1)

In [ ]:
df['product_specifications'][0]

In [ ]:
df['description']

## 1 Data extraction

### 1.1 Categories

Unfolding categories using the 'product_category_tree' colum

In [ ]:
# sample checking
df['product_category_tree'][743]

In [ ]:
# determining the maximum tree depth of categories
ser_depth = df['product_category_tree'].apply(lambda x: x.count('>>'))
max_depth = ser_depth.max()
max_depth

In [ ]:
# Converting the strings in 'product_category_tree' column in 6 categ columns

def str_cleaning(ind, my_str, name_level_cols):
    my_str = my_str.replace("[\"", "").replace("\"]", "")
    tab_str = my_str.split(">>")
    size_tab_str = len(tab_str)
    tup_str = tuple([tab_str[i].strip() if i<size_tab_str else "" \
                     for i in np.arange(max_depth) ])
    return tup_str

name_level_cols = ['cat_level_'+str(i) for i in np.arange(max_depth)]
ser_tuple = df['product_category_tree']\
    .apply(lambda s: str_cleaning(s.index, s, name_level_cols))
df_cat_level = pd.DataFrame([[a,'/'.join([a,b]),'/'.join([a,b,c]),
                              '/'.join([a,b,c,d]),'/'.join([a,b,c,d,e]),
                              '/'.join([a,b,c,d,e,f])] \
                             for a,b,c,d,e,f in ser_tuple.values],
                            columns=name_level_cols, index=df.index)

In [ ]:
# printing number of categories in each level and a sample
display(df_cat_level.nunique(), df_cat_level.sample(3))

Let's see how much items are in each category

In [ ]:
fig = plt.figure(figsize=(25,4))
for i, col in enumerate(df_cat_level.columns,1):
    ax = fig.add_subplot(1,len(df_cat_level.columns), i)
    ser = df_cat_level.groupby(col).size().sort_values(ascending=False)
    ser[0:20].plot.bar(width=0.75, color='grey', ec='k', ax=ax)
    ax.set_title(col+f' ({ser.shape[0]} categories)', fontweight='bold')

The only level that has a balanced set of items is level 0, with 7 categories.
Let's rename these 7 categories:

In [ ]:
df_cat_level['cat_level_0'].unique()

In [ ]:
df_cat_level['category'] = \
    df_cat_level['cat_level_0'].replace({'Home Furnishing': 'Furnishing',
                                        'Baby Care': 'Baby', 
                                        'Watches': 'Watches',
                                        'Home Decor & Festive Needs': 'Decor',
                                        'Kitchen & Dining': 'Kitchen',
                                        'Beauty and Personal Care': 'Beauty',
                                        'Computers': 'Computers'})

### 1.2 Products descriptions

In [ ]:
# extracting only useful data
df_desc_cat = pd.concat([df_cat_level['category'],
                         df[["product_name", "description"]]], axis=1)

In [ ]:
df_desc_cat.head()

In [ ]:
# creation of a corpus of all the descriptions
corpus = ' '.join(df_desc_cat['description'].values)
print("total nb of words in the whole corpus: ", len(corpus.split()))

## 2 Whole corpus exploration

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

#### Frequency of words (with stopwords)

In [ ]:
# tokenizing the words in the whole corpus
tokenizer = nltk.RegexpTokenizer(r'[a-z]+')
li_words = tokenizer.tokenize(corpus.lower())
# counting frequency of each word
ser_freq = pd.Series(nltk.FreqDist(li_words))
# plotting the most frequent words
ser_freq.sort_values(ascending=False)[0:100].plot.bar(width=0.8,
                                                            color='grey',
                                                            ec='k')
plt.gcf().set_size_inches(20,3)

In [ ]:
english_sw = nltk.corpus.stopwords.words('english')
most_freq_sw = (ser_freq.sort_values(ascending=False)[0:36].index).tolist()
most_freq_sw = [s for s in most_freq_sw if s not in english_sw]
single_let_sw = list(string.ascii_lowercase)
single_let_sw = [s for s in single_let_sw if (s not in most_freq_sw) and\
                                             (s not in english_sw)]

#### Frequency of words without stopwords

In [ ]:
# creating a list of stopwords...
sw = set()
# ...including the 36 most frequent words ...
most_freq = (ser_freq.sort_values(ascending=False)[0:36].index).tolist()
sw.update(most_freq)
print("Most frequent words added to stopwords: ", most_freq)
# ...including the single letters ...
single_letters = list(string.ascii_lowercase)
sw.update(single_letters)
# ... as well a known english stopwords from nltk package
sw.update(tuple(nltk.corpus.stopwords.words('english')))
print("Total number of stopwords in sw list: ", len(sw))

In [ ]:
ser_freq_wo_sw = ser_freq.loc[[s for s in ser_freq.index if s not in sw]]

In [ ]:
# plotting the most frequent words
print("o---Total number of words: {}\no---Words in decreasing order of\
 frequency:\n{}".format(ser_freq_wo_sw.sum(),
                        ser_freq_wo_sw.sort_values(ascending=False)))
ser_freq_wo_sw.sort_values(ascending=False)[0:100].plot.bar(width=0.8,
                                                            color='grey',
                                                            ec='k')
plt.gcf().set_size_inches(20,3)

#### Frequency of stems

In [ ]:
## Lemmatization
# WordNetLemmatizer

In [ ]:
from nltk.stem.snowball import EnglishStemmer
stemmer = EnglishStemmer()
ser_freq_stems = pd.Series(ser_freq_wo_sw.values,
                           index=[stemmer.stem(s) for s in ser_freq_wo_sw.index])\
                   .to_frame().reset_index().groupby('index').sum()
ser_freq_stems = pd.Series(ser_freq_stems.iloc[:,0].values,
                           index = ser_freq_stems.index)

In [ ]:
# plotting the most frequent stems
print("o---Total number of words: {}\no---Words in decreasing order of\
 frequency:\n{}".format(ser_freq_stems.sum(),
                        ser_freq_stems.sort_values(ascending=False)))
ser_freq_stems.sort_values(ascending=False)[0:100].plot.bar(width=0.8,
                                                            color='grey',
                                                            ec='k')
plt.gcf().set_size_inches(20,3)

## 3 Text pre-processing


### 3.1 Tokenization - stopwords cleaning - stemming

- regex selection of alphabetical data
- removing stopwords
- removing 36 most frequent words
- stemming

In [ ]:
''' from a sentence, containing words (document):
- tokenizes the words if only composed of alphanumerical data,
- removes stopwords if list is given (stopwords)
- stems the words if stemmer given
NB: This pre-processing function can be used to prepare data for Word2Vec
'''
from nltk.stem.snowball import EnglishStemmer

def tokenize_clean(document, stopwords=None, stemmer=None):
    # tokenizing the words in each description
    tokenizer = nltk.RegexpTokenizer(r'[a-z]+')
    li_words = tokenizer.tokenize(document.lower())
    if stopwords is None: stopwords=[]
    # stemming and removing stopwords
    if stemmer is not None:
        li_words = [stemmer.stem(s) for s in li_words if s not in stopwords]
    else:
        # removing stopwords only
        li_words = [s for s in li_words if s not in stopwords]
    return li_words

In [ ]:
from nltk.stem.snowball import EnglishStemmer
stemmer = EnglishStemmer()
df_desc_cat['desc_token'] = \
    df_desc_cat['description'].apply(lambda x: tokenize_clean(x,
                                                              stopwords=sw,
                                                              stemmer=stemmer))

In [ ]:
df_desc_cat['desc_token_joined'] = \
    df_desc_cat['desc_token'].apply(lambda x: ' '.join(x))

In [ ]:
print(df_desc_cat.shape[0], "different descriptions")
df_desc_cat.head()

In [ ]:
big_list = list(df_desc_cat['desc_token'].values)
all_terms = [s for sublist in big_list for s in sublist]
print("Total number of words in all the descriptions (with duplicates) :",
      len(all_terms))
print("Total number of words in all the descriptions (unique) :",
      len(set(all_terms)))

### 3.2 Document-Term Matrices (BOW and TF-IDF)

#### Bag of words using CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(stop_words = sw,
                    #   min_df = 3,
                    #   max_df = 70,
                      max_features = 1600,
                      )

CV_TD_mat = vec.fit_transform(df_desc_cat['description'])
print( "Created %d X %d document-term matrix" % (CV_TD_mat.shape[0],
                                                 CV_TD_mat.shape[1]) )

# Vocabulary of the document_term matrix
CV_voc = vec.get_feature_names()
print("Vocabulary has %d distinct terms" % len(CV_voc))

CV_TD_df = pd.DataFrame(CV_TD_mat.todense(),
             index=df_desc_cat.index, # each item
             columns=CV_voc) # each word

In [ ]:
# New Stopwords list generated because of CountVectorizer parameters
CV_sw = vec.stop_words_
print("Old stop-words list has %d entries" % len(sw) )
print("New stop-words list has %d entries" % len(CV_sw))

Let's look at a representation of the categories on a projection of the data

In [ ]:
# BOW projection

ser_categories = pd.Series(df_desc_cat['category'].values, 
                           index=df_desc_cat.index,
                           name='Clust')

plot_projection(CV_TD_df, model=None, ser_clust=ser_categories,
                proj='t-SNE', size=10, legend_on=True,
                title="Projection t-SNE of the documents\n(BOW)",
                figsize=(10, 6), palette='tab10',
                fig=None, ax=None, random_state=14)

#### TF_IDF matrix using TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer(stop_words=sw,
                            #   min_df = 3,
                            #   max_df = 70,
                            max_features = 1600
                            )
TFIDF_TD_mat = tfidf_vec.fit_transform(df_desc_cat['description'])
print( "Created %d X %d TF-IDF-normalized document-term matrix"\
      % (TFIDF_TD_mat.shape[0], TFIDF_TD_mat.shape[1]) )

# Vocabulary of the document_term matrix
TFIDF_voc = tfidf_vec.get_feature_names()
print("Vocabulary has %d distinct terms" % len(TFIDF_voc))

TFIDF_TD_df = pd.DataFrame(TFIDF_TD_mat.todense(),
                           index=df_desc_cat.index, # each item
                           columns=TFIDF_voc) # each word

In [ ]:
# New Stopwords list genearated because of CountVectorizer parameters
TFIDF_sw = tfidf_vec.stop_words_
print("Old stop-words list has %d entries" % len(sw) )
print("New stop-words list has %d entries" % len(TFIDF_sw))

Let's look at a representation of the categories on a projection of the data

In [ ]:
# TF-IDF projection

ser_categories = pd.Series(df_desc_cat['category'].values, 
                           index=df_desc_cat.index,
                           name='Clust')

plot_projection(TFIDF_TD_df, model=None, ser_clust=ser_categories,
                proj='t-SNE', size=10, legend_on=True,
                title="Projection t-SNE of the documents\n(Tf-Idf matrix)",
                figsize=(10, 6), palette='tab10',
                fig=None, ax=None, random_state=14)

### 3.3 Word embeddings

#### Wiki2Vec embedding

Let's use a word embedding to get a smarter and smaller representation of our data.
We'll use a pre-trained Word2Vec model (our data are too small to train a model).

In [ ]:
from wikipedia2vec import Wikipedia2Vec

In [ ]:
## Option 2: using a pretrained model
# https://wikipedia2vec.github.io/wikipedia2vec/pretrained/


In [ ]:
wiki2vec = Wikipedia2Vec.load("../DATA/enwiki_20180420_100d.pkl") # 100 dimensions

In [ ]:
# Vectorization of the descriptions prior to projection on Wiki2Vec

# from nltk.stem.snowball import EnglishStemmer
# stemmer = EnglishStemmer()

tf_idf_df, actu_sw = \
    compute_doc_terms_df(df_desc_cat['description'],
                         preproc_func= tokenize_clean,
                         preproc_func_params = {'stopwords': sw,
                                                'stemmer': None},
                         vec_params = {'min_df': 1,
                                    #    'max_df': 15,
                                       'stop_words': sw},
                         tfidf_on=True)

In [ ]:
# Projection of the document_term matrix on the w2v matrix 

wiki2vec_emb_df = proj_term_doc_on_w2v(tf_idf_df, wiki2vec, print_opt=False)

Let's look at a representation of the categories on a projection of the data

In [ ]:
# tfidf_Wiki2Vec_emb projection

ser_categories = pd.Series(df_desc_cat['category'].values, 
                           index=df_desc_cat.index,
                           name='Clust')

plot_projection(wiki2vec_emb_df, model=None, ser_clust=ser_categories,
                proj='t-SNE', size=10, legend_on=True,
                title="Projection t-SNE of the documents\n(wiki2vec embedding)",
                figsize=(10, 6), palette='tab10',
                fig=None, ax=None, random_state=14)

#### Training a Word2Vec model on the corpus (gensim)

In [ ]:
## Option 1: training a Word2Vec model on the whole corpus

from gensim.models.word2vec import Word2Vec

cust_w2v = Word2Vec(df_desc_cat['desc_token'], # works on lists of tokenized stemmed words
                 size=300,
                 window=20,
                 min_count=2,
                 workers=1,
                 iter=100,
                 sg=1)  #0: CBOW, 1:skip-gram)
cust_w2v_vocab = cust_w2v.wv.vocab
print( "Model has %d terms" % len(cust_w2v_vocab))

# pickle the model
cust_w2v.save('P6_trained_w2v.bin')

# # reload
# w2v_model = gensim.models.Word2Vec.load("w2v-model.bin")

In [ ]:
# Vectorization of the descriptions prior to projection on Wiki2Vec

# from nltk.stem.snowball import EnglishStemmer
# stemmer = EnglishStemmer()

tf_idf_df, actu_sw = \
    compute_doc_terms_df(df_desc_cat['description'],
                         preproc_func= tokenize_clean,
                         preproc_func_params = {'stopwords': sw,
                                                'stemmer': None},
                         vec_params = {'min_df': 1,
                                    #    'max_df': 15,
                                       'stop_words': sw},
                         tfidf_on=True)

In [ ]:
# examples
cust_w2v.wv['polyest'].shape, cust_w2v.wv['polyest'][:10]

In [ ]:
tf_idf_df.shape, cust_w2v.wv.vectors.shape

In [ ]:
## A REPARER (ATTRIBUT GET_WORD)
# # Projection of the document_term matrix on the w2v matrix 

# custom_w2v_emb_df = proj_term_doc_on_w2v(tf_idf_df, cust_w2v, print_opt=True)

# # projection of the Document_terms matrix on the wiki2vec (manually, as the vocabulary is the same)
# w2v_emb_df = tf_idf_df.dot(custom_w2v_emb_df)

Let's look at a representation of the categories on a projection of the data

In [ ]:
# # tfidf_Word2Vec_emb projection

# ser_categories = pd.Series(df_desc_cat['category'].values, 
#                            index=df_desc_cat.index,
#                            name='Clust')

# plot_projection(custom_w2v_emb_df, model=None, ser_clust=ser_categories,
#                 proj='t-SNE', size=10, legend_on=True,
#                 title="Projection t-SNE of the documents\n(wiki2vec embedding)",
#                 figsize=(10, 6), palette='tab10',
#                 fig=None, ax=None, random_state=14)

## 4 Visualization of the vectorizations with respect to the known categories

In [ ]:
'''
'params' are the arguments of the 'compute_doc_terms_df' function and
take the form (default settings):
    preproc_func=None,
    preproc_func_params=None,
    vec_params = {'min_df': 1},
    tfidf_on=False,
    print_opt=False
'''

from sklearn.metrics import silhouette_score

def plot_grid_proj_params(ser_desc, ser_clust, params, n_rows=1,
                          w2v=None, title=None, figsize=(20,10)): 

    n_cols = len(params)//n_rows + ((len(params)%n_rows)>0)*1

    fig = plt.figure()
    fig.set_size_inches(figsize)

    df_scores = pd.DataFrame()
    # Loops over the parameter to show the projections
    for i, param in enumerate(params,1):

        ax = fig.add_subplot(n_rows, n_cols, i)
        # Vectorization of the descriptions prior to projection on Wiki2Vec
        doc_term_df, actu_sw = compute_doc_terms_df(ser_desc, **param)

        # Projection of the document_term matrix on the w2v matrix
        if w2v is not None:
            doc_term_df = proj_term_doc_on_w2v(doc_term_df, w2v)

        # Computing scores
        silh = silhouette_score(doc_term_df, ser_clust)
        df_scores.loc[i-1,'silh'] = silh

        # tfidf_Wiki2Vec_emb projection
        plot_projection(doc_term_df, model=None, ser_clust=ser_clust,
                        proj='t-SNE', size=10, legend_on=True,
                        title="param set n°{}|silh.={:.2f}".format(i, silh),
                        fig=fig, ax=ax, random_state=14)
        
    if title:
        plt.suptitle(title, fontsize=16, fontweight='bold')
        plt.tight_layout(rect=(0,0,1,0.92))
    else:
        plt.tight_layout()
    plt.show()
    return df_scores

Document-terms matrices (BOW and TfIdf)

In [ ]:
# ####### BOW: min_df#######
# ser_categories = pd.Series(df_desc_cat['category'].values, 
#                            index=df_desc_cat.index,
#                            name='Clust')

# # Generates a list of parameters dictionaries
# min_df_range = (np.linspace(0,60,6)).astype('int')
# params = []
# for min_df in min_df_range:
#     params.append({'vec_params': {'min_df': min_df, 'stop_words': sw},
#                    'tfidf_on': False})

# print(f"Range of min_df tested:\n{min_df_range}")
# df_scores = plot_grid_proj_params(df_desc_cat['description'],
#                                   ser_categories,
#                                   params, n_rows=2,
#                                   figsize=(15,7),
#                                   w2v=None, title="Bag of Word")

# display(df_scores.T)

In [ ]:
# ####### TF_IDF: min_df#######
# ser_categories = pd.Series(df_desc_cat['category'].values, 
#                            index=df_desc_cat.index,
#                            name='Clust')

# # Generates a list of parameters dictionaries
# min_df_range = (np.linspace(0,60,6)).astype('int')
# params = []
# for min_df in min_df_range:
#     params.append({'vec_params': {'min_df': min_df, 'stop_words': sw},
#                    'tfidf_on': True})

# print(f"Range of min_df tested:\n{min_df_range}")
# df_scores = plot_grid_proj_params(df_desc_cat['description'],
#                                   ser_categories,
#                                   params, n_rows=2,
#                                   figsize=(15,7),
#                                   w2v=None, title='TF-IDF matrix')

# display(df_scores.T)

Document-terms matrices (BOW and TfIdf) with W2V projection

In [ ]:
# ####### BOW->W2V: min_df#######
# ser_categories = pd.Series(df_desc_cat['category'].values, 
#                            index=df_desc_cat.index,
#                            name='Clust')

# # Generates a list of parameters dictionaries
# min_df_range = (np.linspace(0,60,6)).astype('int')
# params = []
# for min_df in min_df_range:
#     params.append({'vec_params': {'min_df': min_df, 'stop_words': sw},
#                    'tfidf_on': False})

# print(f"Range of min_df tested:\n{min_df_range}")
# df_scores = plot_grid_proj_params(df_desc_cat['description'],
#                                   ser_categories,
#                                   params, n_rows=2,
#                                   figsize=(15,7),
#                                   w2v=wiki2vec,
#                                   title='Bag of Words -> Wiki2Vec')

# display(df_scores.T)

In [ ]:
# ####### BOW->W2V: min_df#######
# ser_categories = pd.Series(df_desc_cat['category'].values, 
#                            index=df_desc_cat.index,
#                            name='Clust')

# # Generates a list of parameters dictionaries
# min_df_range = (np.linspace(0,60,6)).astype('int')
# params = []
# for min_df in min_df_range:
#     params.append({'vec_params': {'min_df': min_df, 'stop_words': sw},
#                    'tfidf_on': True})

# print(f"Range of min_df tested:\n{min_df_range}")
# df_scores = plot_grid_proj_params(df_desc_cat['description'],
#                                   ser_categories,
#                                   params, n_rows=2,
#                                   figsize=(15,7),
#                                   w2v=wiki2vec,
#                                   title='TF-IDF Matrix-> Wiki2Vec')

# display(df_scores.T)

## 5 Topics Modeling

We'll create a dataframe where we'll put the results of the best results of clustering or topics modeling obtained with different techniques.

In [ ]:
df_res_clust = pd.DataFrame()
df_res_clust['categories'] = df_desc_cat['category']

### 5.1 Categories Modeling using NMF

In [ ]:
# create the model
from sklearn.decomposition import NMF
k=7 # number of categories
nmf = NMF(init="nndsvd",
          n_components=k) 

Preprocessing and vectorization of the data

In [ ]:
# Vectorization of the descriptions prior to applying NMF

tf_idf_df, _ = \
    compute_doc_terms_df(df_desc_cat['description'],
                         preproc_func= tokenize_clean,
                         preproc_func_params = {'stopwords': sw,
                                                'stemmer': None},
                         vec_params = {'min_df': 10,
                                       'stop_words': sw},
                         tfidf_on=True)

Let's have a look to the 2 matrices documents/topics ans topics/words

In [ ]:
# DOCUMENTS/TOPICS Matrix
W = pd.DataFrame(nmf.fit_transform(tf_idf_df.values),
                 index=tf_idf_df.index, # documents
                 columns=['clust_'+str(i) for i in range(1,k+1)]) # topics
# TOPICS/WORDS Matrix
H = pd.DataFrame(nmf.components_,
                 index=['clust_'+str(i) for i in range(1,k+1)], # topics
                 columns=tf_idf_df.columns) # words
print(W.shape, H.shape)

#### Categories descriptors

Let's have a look to the top words for each category.

In [ ]:
plot_wordclouds_topwords(H, n_top_words=20, n_rows=2)

In [ ]:
# printing 10 top words for each topic
for i in H.index:
    print(i, '---', H.loc[i].sort_values(ascending=False)[0:10].index.tolist())
    print(i, '---',
          np.round(H.loc[i].sort_values(ascending=False)[0:10].values.tolist(),2))

In [ ]:
# plotting bar plots fo 20 top words for each topic
n_rows = 2
fig= plt.figure(figsize=(15,5))
for i, cat in enumerate(H.index, 1):
    n_tot = H.shape[0]
    n_cols = n_tot//n_rows + (n_tot%n_rows!=0)*1
    ax=fig.add_subplot(n_rows, n_cols, i)
    H.loc[cat].sort_values(ascending=False).iloc[0:20].plot.bar(color='grey',
                                                                width=0.8,
                                                                ec='k', ax=ax)
    ax.set_title(cat, fontweight='bold')
plt.tight_layout()
plt.show()

#### Most relevant items for each category

In [ ]:
# 3 most relevant items for each categories
categories = ['clust_'+str(i) for i in range(1,8)]
for cat in categories:
    ser = W.loc[:,cat].sort_values(ascending=False)[0:3]
    print('-'*20+cat.upper()+'-'*20)
    display(df_desc_cat.loc[ser.index, ['product_name', 'description']])

In [ ]:
# Appending the best results of the NMF topics modeling 
df_res_clust['NMF_tfidf'] = W.idxmax(1)

In [ ]:
df_res_clust.sample(5)

### 5.2 Categories modeling with LDA

In [ ]:
# create the model
from sklearn.decomposition import LatentDirichletAllocation as LDA
k=7 # number of categories

lda = LDA(n_components=k)

Preprocessing and vectorization of the data

In [ ]:
# Vectorization of the descriptions prior to applying LDA
tf_idf_df, _ = \
    compute_doc_terms_df(df_desc_cat['description'],
                         preproc_func= tokenize_clean,
                         preproc_func_params = {'stopwords': sw,
                                                'stemmer': None},
                         vec_params = {'min_df': 10,
                                       'stop_words': sw},
                         tfidf_on=True)

Let's have a look to the 2 matrices documents/topics ans topics/words

In [ ]:
# DOCUMENTS/TOPICS Matrix
W = pd.DataFrame(lda.fit_transform(tf_idf_df.values),
                 index=tf_idf_df.index, # documents
                 columns=['clust_'+str(i) for i in range(1,k+1)]) # topics
# TOPICS/WORDS Matrix
H = pd.DataFrame(lda.components_,
                 index=['clust_'+str(i) for i in range(1,k+1)], # topics
                 columns=tf_idf_df.columns) # words
print(W.shape, H.shape)

#### Categories descriptors

Let's have a look to the top words for each category.

In [ ]:
plot_wordclouds_topwords(H, n_top_words=20, n_rows=2)

In [ ]:
# printing 10 top words for each topic
for i in H.index:
    print(i, '---', H.loc[i].sort_values(ascending=False)[0:10].index.tolist())
    print(i, '---',
          np.round(H.loc[i].sort_values(ascending=False)[0:10].values.tolist(),2))

In [ ]:
# plotting bar plots fo 20 top words for each topic
n_rows = 2
fig= plt.figure(figsize=(15,5))
for i, cat in enumerate(H.index, 1):
    n_tot = H.shape[0]
    n_cols = n_tot//n_rows + (n_tot%n_rows!=0)*1
    ax=fig.add_subplot(n_rows, n_cols, i)
    H.loc[cat].sort_values(ascending=False).iloc[0:20].plot.bar(color='grey',
                                                                width=0.8,
                                                                ec='k', ax=ax)
    ax.set_title(cat, fontweight='bold')
plt.tight_layout()
plt.show()

#### Most relevant items for each category

In [ ]:
# 10 most relevant items for each categories
categories = ['clust_'+str(i) for i in range(1,8)]
for cat in categories:
    ser = W.loc[:,cat].sort_values(ascending=False)[0:10]
    print('-'*20+cat.upper()+'-'*20)
    display(df_desc_cat.loc[ser.index, ['product_name', 'description']])

In [ ]:
# Appending the best results of the LDA topics modeling 
df_res_clust['LDA_tfidf'] = W.idxmax(1)

In [ ]:
df_res_clust.sample(5)

### 5.3 Categories modeling with LSA

In [ ]:
# create the model
from sklearn.decomposition import TruncatedSVD
k=7 # number of categories

lsa = TruncatedSVD(n_components=k)

Preprocessing and vectorization of the data

In [ ]:
# Vectorization of the descriptions prior to applying LDA
tf_idf_df, _ = \
    compute_doc_terms_df(df_desc_cat['description'],
                         preproc_func= tokenize_clean,
                         preproc_func_params = {'stopwords': sw,
                                                'stemmer': None},
                         vec_params = {'min_df': 10,
                                       'stop_words': sw},
                         tfidf_on=True)

Let's have a look to the 2 matrices documents/topics ans topics/words

In [ ]:
# DOCUMENTS/TOPICS Matrix
W = pd.DataFrame(lsa.fit_transform(tf_idf_df.values),
                 index=tf_idf_df.index, # documents
                 columns=['clust_'+str(i) for i in range(1,k+1)]) # topics
# TOPICS/WORDS Matrix
H = pd.DataFrame(lda.components_,
                 index=['clust_'+str(i) for i in range(1,k+1)], # topics
                 columns=tf_idf_df.columns) # words
print(W.shape, H.shape)

#### Categories descriptors

Let's have a look to the top words for each category.

In [ ]:
plot_wordclouds_topwords(H, n_top_words=20, n_rows=2)

In [ ]:
# printing 10 top words for each topic
for i in H.index:
    print(i, '---', H.loc[i].sort_values(ascending=False)[0:10].index.tolist())
    print(i, '---',
          np.round(H.loc[i].sort_values(ascending=False)[0:10].values.tolist(),2))

In [ ]:
# plotting bar plots fo 20 top words for each topic
n_rows = 2
fig= plt.figure(figsize=(15,5))
for i, cat in enumerate(H.index, 1):
    n_tot = H.shape[0]
    n_cols = n_tot//n_rows + (n_tot%n_rows!=0)*1
    ax=fig.add_subplot(n_rows, n_cols, i)
    H.loc[cat].sort_values(ascending=False).iloc[0:20].plot.bar(color='grey',
                                                                width=0.8,
                                                                ec='k', ax=ax)
    ax.set_title(cat, fontweight='bold')
plt.tight_layout()
plt.show()

#### Most relevant items for each category

In [ ]:
# 10 most relevant items for each categories
categories = ['clust_'+str(i) for i in range(1,8)]
for cat in categories:
    ser = W.loc[:,cat].sort_values(ascending=False)[0:3]
    print('-'*20+cat.upper()+'-'*20)
    display(df_desc_cat.loc[ser.index, ['product_name', 'description']])

In [ ]:
# Appending the best results of the LDA topics modeling 
df_res_clust['LSA_tfidf'] = W.idxmax(1)

In [ ]:
df_res_clust.sample(5)

### 5.4 KMeans Clustering

#### Clustering on BOW and TF-IDF matrices

In [ ]:
# Create the model
from sklearn.cluster import KMeans
k=7 # number of categories
km = KMeans(n_clusters=k) 

Preprocessing and vectorization of the data

In [ ]:
# Vectorization of the descriptions prior to projection on Wiki2Vec

tf_idf_df, _ = \
    compute_doc_terms_df(df_desc_cat['description'],
                         preproc_func= tokenize_clean,
                         preproc_func_params = {'stopwords': sw,
                                                'stemmer': None},
                         vec_params = {'min_df': 10,
                                       'stop_words': sw},
                         tfidf_on=True)

In [ ]:
# Fitting the Kmeans model
km.fit(tf_idf_df)

In [ ]:
# Appending the best results of the Kmeans clustering
df_res_clust['KMclust_tfidf'] = km.labels_

In [ ]:
df_res_clust.sample(5)

#### Clustering on Word2vec embedding

#### Clustering on NMF selected features

#### Clustering on LDA selected features

### 5.5 Comparison of clusters labels with true categories

#### ARI scores (true categories vs. clusters)

In [ ]:
from sklearn.metrics import adjusted_rand_score

In [ ]:
df_res_clust.columns

In [ ]:
adjusted_rand_score(df_res_clust['categories'],
                    df_res_clust['NMF_tfidf'])

In [ ]:
ser_ari_pairs_models = ARI_column_pairs(df_res_clust, first_vs_others=True,
                                        print_opt=True)

In [ ]:
fig = plt.figure(figsize=(2,3))
ser_ari_pairs_models.plot.bar(width=0.7, color='grey', ec='k')
plt.ylabel('ARI score')
# plt.title('ARI score comparing the cluster\nlabel prediction of pairs of models')
# plt.gca().set(ylim=(0.85,1))
plt.show()

#### Confusion matrix

In [ ]:
cat_clust_confmat = confusion_matrix_clust(df_res_clust['categories'],
                                           df_res_clust['NMF_tfidf'],
                                           normalize=False,
                                           margins_sums=True,
                                           margins_score=True)
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(cat_clust_confmat)

In [ ]:
cm = confusion_matrix_clust(df_res_clust['categories'],
                            df_res_clust['NMF_tfidf'],
                            normalize=False,
                            margins_sums=False,
                            margins_score=False)
plot_heatmap(cm, "Confusion matrix | true categories vs. clusters",
             figsize=(8, 4), vmin=0, vmax=150, center=75,
                 palette=sns.color_palette("viridis", 20), shape='rect',
                 fmt='.0f', robust=False, fig=None, ax=None)

####  Sankey confusion diagram

In [ ]:
dict_link_color = dict(zip(range(n_cat),
                           sns.color_palette(palette, cm.shape[0]).as_hex()))


In [ ]:
import plotly.graph_objects as go
from IPython.display import Image

'''
Takes a confusion matrix (best if diagonal maximized) with true categories as
indices and clusters as columns (can be obtained using the function
'confusion_matrix_clust', which ensures the diagonal values are maximum i.e.
the best bijective correponding cat-clut pairs have been found),
then plot the sankey confusion matrix.
Use the option static to plot a static image of the original interactive graph.

NB: the code below needs to be run if you are on colab

    # in order to get orca to be installed in colab (to display static plotly graphs)
    !pip install plotly>=4.0.0
    !wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
    !chmod +x /usr/local/bin/orca
    !apt-get install xvfb libgtk2.0-0 libgconf-2-4
'''

def plot_sankey_confusion_mat(cm, static=False, figsize=(2, 1.7),
                              font_size=14, scale=1):

    n_cat = cm.shape[0]
    n_clust = cm.shape[1]
    source = np.array([n_clust*[i] for i in range(n_cat)]).ravel()
    target = np.array([[i] for i in range(n_cat,n_clust+n_cat)]*n_cat).ravel()
    value = cm.values.ravel()
    nodes_lab = list(cm.index)+list(cm.columns)
    palette = 'tab10'
    alpha_nodes, alpha_links = 0.7, 0.3
    my_pal = sns.color_palette(palette, max(cm.shape))
    pal_nodes_cat = list([f'rgba({r},{g},{b},{alpha_nodes})' \
                        for r, g, b in my_pal[:n_cat]])
    pal_nodes_clust = list([f'rgba({r},{g},{b},{alpha_nodes})' \
                            for r, g, b in my_pal[:n_clust]])
    nodes_colors = (pal_nodes_cat + pal_nodes_clust)#.as_hex()

    pal_links = list([f'rgba({r},{g},{b},{alpha_links})' for r, g, b in my_pal[:n_cat]])
    dict_links_colors = dict(zip(range(n_cat), pal_links))
    links_colors = np.vectorize(dict_links_colors.__getitem__)(source)#.as_hex()

    # Prepare the graph
    fig = go.Figure(data=[go.Sankey(node = dict(pad = 15,
                                                thickness = 20,
                                                line = dict(color = "black",
                                                            width = 0.5),
                                                label = nodes_lab,
                                                color = nodes_colors),
                                    link = dict(source = source,
                                                target = target,
                                                value = value,
                                                # label = ,
                                                color = links_colors,
                                                ))])
    # title
    fig.update_layout(title_text="Sankey confusion diagram | \
true categories vs. clusters", font_size=font_size)
    if static:
        w, h = figsize
        img_bytes = fig.to_image(format="png", width=w, height=h, scale=scale)
        # Image(img_bytes)
        return img_bytes
    else:
        fig.show()

In [ ]:
# interactive
plot_sankey_confusion_mat(cm, static=False,)
                      #   font_size=14, figsize=(2, 1.7), scale = 1)

In [ ]:
# static
from IPython.display import Image
plot_sankey_confusion_mat(cm, static=False,)
                      #   font_size=14, figsize=(2, 1.7), scale = 1)
img_bytes

#### 2D visualisation

In [ ]:
# Plotting a projection of the points (true categories vs. clusters) (PCA 0.71, UMAP 0.81, t-SNE 0.83)
ind_samp = df_res_clust.sample(200).index
plot_projection_cat_clust(CV_TD_df.loc[ind_samp], model=None,
                 ser_clust=df_res_clust['NMF_tfidf'].loc[ind_samp], # clustering already done
                 true_cat=df_res_clust['categories'].loc[ind_samp], # true categories
                 proj='t-SNE', size=35, edgelinesize=3, legend_on=True,
                 title="True categories vs. Clusters ( BOW -> t-SNE )",
                 figsize=(8, 5), palette='tab10',
                 fig=None, ax=None, random_state=14, bboxtoanchor=(1,1))

- calcul de la pertinence des catégories trouvées par rapport aux catégories initiales et représentation graphique

- optimisation du nombre de catégories par mesure de la cohérence des mots les plus représentatifs de chaque catégorie (nécessite d'entraîner un modèle word2vec)
- représentation des catégories (des mots les plus représentatifs qui les composent) avec t-SNE
- essai direct de clustering sur la représentation des mots par Word2Vec

- Latent Dirichlet Allocation pour trouver d'autres clusters

- comment mettre en place un vote permettant d'aggréger le résultat de plusieurs clusterings différents lorsque les catégories n'ont pas les mêmes numéros ? 

### FUNCTIONS

- Ajouter les clusters (autre forme de marqueurs)
- Tracer les centroïdes des clusters
- Trouver une correspondance entre les clusters et les catégories
- une ligne qui relie les clusters

## Exportation

Now we export the dataset of aggregated orders in a .csv file.

In [ ]:
dfs['ord_it'].to_csv('agg_order_items.csv')